## Desafío del carro de la compra

Una tienda online le ha pedido que diseñe su capa de almacenamiento de datos y su(s) tabla(s) NoSQL. El sitio web sirve a los clientes y a los productos que ven, guardan y compran. El tráfico del sitio web es actualmente bajo, pero quieren ser capaces de servir a millones de clientes simultáneos.

* Los clientes interactúan con productos que pueden estar `ACTIVE`, `SAVED` o `PURCHASED`. Una vez que son `PURCHASED` se les asigna un OrderId.
* Los productos tienen los siguientes atributos: AccountID, Status (ACTIVE, SAVED, or PURCHASED), CreateTimestamp, and ItemSKU (El tamaño total del artículo es <= 1 KB).
* Cuando un cliente abre la aplicación de la tienda, ve los productos activos en su cesta, que están organizados por los añadidos más recientemente.
* Los usuarios pueden ver los productos que han guardado para más tarde, organizados por los últimos guardados.
* Los usuarios pueden ver los productos que han comprado, organizados por los últimos comprados.
* Los equipos de producto tienen la capacidad de consultar regularmente a todos los clientes para identificar a las personas que tienen un producto específico en su cuenta que está `ACTIVE`, `SAVED` o `PURCHASED`.
* El equipo de Business Intelligence necesita ejecutar una serie de consultas ad hoc complejas en el conjunto de datos para crear informes semanales y mensuales.

Construir un Modelo de Datos NoSQL para cumplir con el componente OLTP (OnLine Transaction Processing) de la carga de trabajo. ¿Cómo cumpliría los requisitos del equipo de BI?

#### ¿Cuáles son los patrones de acceso?

**Los patrones de acceso en el escenario son:**

* Insertar y actualizar artículos colocados en un carro por los usuarios.
* Devuelve los artículos relacionados con un usuario (AccountID), ordenados por CreateTimestamp y asignados a un estado específico.
* Devolución de artículos a través del usuario por ItemSKU, ordenados por CreateTimestamp, y con alcance a un Estado específico.
* Consultas ad hoc fuera de línea para el equipo de Business Intelligence.

**Identifique posibles claves de partición para cumplir el patrón de acceso primario:**

* ¿Qué atributo de ítem escala en volumen junto con un mayor acceso?
* ¿Cuál es la organización natural de los elementos de datos relacionados (para devolver los elementos recopilados en relación con los patrones de acceso anteriores)?
* Considere la dimensión del acceso: tanto lecturas como escrituras.

**Al determinar cómo organizar los elementos relacionados con el patrón de acceso principal:**

* ¿Qué organización debe escribirse para devolver los elementos ordenados (ordenar por)?
* ¿Cuál es la jerarquía de las relaciones (de más general a más específica)?

**Cumplimiento de los patrones de acceso segundo, tercero y cuarto:**

* Los patrones de acceso segundo y tercero son patrones de acceso OLTP y pueden modelarse directamente en DynamoDB.
* El cuarto patrón de acceso es OLAP y no necesita cumplirse directamente en DynamoDB, o en su solución para el caso.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
import time
from decimal import Decimal

In [8]:
dt = DynamoTable(profile_name='089715336747_DynamoAttributes')
#dt=DynamoTable()
try:
    dt.select_table('CustomerPurchases')
    print(dt)
except:
    dt.create_table(
        table_name='CustomerPurchases',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
    )

Table created successfully!


In [3]:
# Create Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1PK",
    att_type="S",
    sort_index="GSI1SK",
    sort_type="S",
    i_name="GSI1",
    proj_type=["ItemSKU"]
)

In [4]:
status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 11.06 minute


In [12]:
df = pd.read_csv('retail_cart_chall.csv')
nx = df.ItemSKU.to_list()
new_dt = []
for elem in nx:
    new_dt.append(json.loads(elem, parse_float=Decimal))
df['ItemSKU'] = new_dt
dt.batch_pandas(df)
print("Data successfully loaded into DynamoDB table.")

Data successfully loaded into DynamoDB table.


### Agregar elementos al carro de compras

In [93]:
from datetime import datetime

In [94]:
iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'

dt.add_item(
    item={
        "PK": "12856333",
        "SK": "12856333#ACTIVE#P-8995",
        "AccountId": "12856333",
        "CreateTimestamp": iso_date,
        "GSI1PK": "12856333#ACTIVE",
        "GSI1SK": iso_date,
        "ItemSKU": {"ProductId":"P-8995","Quantity": 2, "Price": Decimal('199.99'), "Category": "Electronics"},
        "Status": "ACTIVE"
    }
) 
    

In [95]:
iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'

dt.add_item(
    item={
        "PK": "12856333",
        "SK": "12856333#ACTIVE#P-8998",
        "AccountId": "12856333",
        "CreateTimestamp": iso_date,
        "GSI1PK": "12856333#ACTIVE",
        "GSI1SK": iso_date,
        "ItemSKU": {"ProductId":"P-8998","Quantity": 1, "Price": Decimal('499.99'), "Category": "Electronics"},
        "Status": "ACTIVE"
    }
) 

In [96]:
iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'

dt.add_item(
    item={
        "PK": "12856333",
        "SK": "12856333#ACTIVE#P-10599",
        "AccountId": "12856333",
        "CreateTimestamp": iso_date,
        "GSI1PK": "12856333#ACTIVE",
        "GSI1SK": iso_date,
        "ItemSKU": {"ProductId":"P-10599","Quantity": 1, "Price": Decimal('699.99'), "Category": "TV"},
        "Status": "ACTIVE"
    }
) 
   

### Obtener todos los elementos del carrito:

In [97]:
response = dt.query("12856333", "12856333#ACTIVE*")

In [98]:
def elem_total(elem):
    total = 0
    for item in elem:
        total += item['ItemSKU']['Quantity']
    return total

def calc_total(items):
    total = 0
    for item in items:
        total += item['ItemSKU']['Price'] * item['ItemSKU']['Quantity']
    return total

def get_category(items):
    category = []
    for item in items:
        category.append(item['ItemSKU']['Category'])
    return list(set(category))

In [99]:
print("Cantidad de elementos:", elem_total(response))
print("Total de la compra:", calc_total(response), "USD")
print("Categorias de los productos:", get_category(response))

Cantidad de elementos: 4
Total de la compra: 1599.96 USD
Categorias de los productos: ['TV', 'Electronics']


### Realizar la compra

In [23]:
# Generate unique id
import random
sample_num = random.randint(100000, 999999)
token = f"O-{sample_num}"

if dt.table_arn.split(':')[4] != '089715336747':
    session = boto3.Session(profile_name='089715336747_DynamoAttributes')
    dynamo_client = session.client('dynamodb', region_name='us-east-1')
    kinesis_client = session.client('kinesis', region_name='us-east-1')
    print("Profile 089715336747_DynamoAttributes")
else:
    dynamo_client = boto3.client('dynamodb', region_name='us-east-1')
    kinesis_client = boto3.client('kinesis', region_name='us-east-1')
    print("Profile default")

Profile 089715336747_DynamoAttributes


In [103]:
action_items = []
iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'

for item in response:
    action_items.append(
        {
            "Put":
                {
                    "TableName": dt.table_name,
                    "Item": {
                        "PK": {"S": item['PK']},
                        "SK": {"S": item['PK']+"#PURCHASED"+"#"+token+"#"+item['ItemSKU']['ProductId']},
                        "AccountId": {"S": item['AccountId']},
                        "CreateTimestamp": {"S": iso_date},
                        "GSI1PK": {"S": item['PK']+"#PURCHASED"},
                        "GSI1SK": {"S": iso_date},
                        "ItemSKU": {"M": {
                            "ProductId": {"S": item['ItemSKU']['ProductId']},
                            "Quantity": {"N": str(item['ItemSKU']['Quantity'])},
                            "Price": {"N": str(item['ItemSKU']['Price'])},
                            "Category": {"S": item['ItemSKU']['Category']}
                            }
                        },
                        "Status": {"S": "PURCHASED"},
                        "OrderId": {"S": token}
                    }
                }
        }
    )

In [104]:
for item in response:
    action_items.append(
        {
            "Delete":
                {
                    "Key": {"PK": {"S": item['PK']}, "SK": {"S": item['SK']}},
                    "TableName": dt.table_name
                }
        }
    )
    

In [105]:
try:
    dynamo_client.transact_write_items(
        TransactItems=action_items,
        ClientRequestToken=token
    )
    print("Transaction successful.")
except ClientError as e:
    print(f"Error: {e.response['Error']['Message']}")

Transaction successful.


In [106]:
response = dt.query("12856333", "12856333#PURCHASED#"+token+"*")
if response:
    for item in response:
        print(item['ItemSKU']['ProductId'], item['ItemSKU']['Quantity'], item['ItemSKU']['Price'], item['ItemSKU']['Category'])

P-10599 1 699.99 TV
P-8995 2 199.99 Electronics
P-8998 1 499.99 Electronics


### Guardar elementos en favoritos

In [142]:
iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'

dt.add_item(
    item={
        "PK": "12856333",
        "SK": "12856333#SAVED#P-8997",
        "AccountId": "12856333",
        "CreateTimestamp": iso_date,
        "GSI1PK": "12856333#SAVED",
        "GSI1SK": iso_date,
        "ItemSKU": {"ProductId":"P-8997", "Category": "TV"},
        "Status": "SAVED"
    }
) 

In [143]:
iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'

dt.add_item(
    item={
        "PK": "12856333",
        "SK": "12856333#SAVED#P-8995",
        "AccountId": "12856333",
        "CreateTimestamp": iso_date,
        "GSI1PK": "12856333#SAVED",
        "GSI1SK": iso_date,
        "ItemSKU": {"ProductId":"P-8995", "Category": "Electronics"},
        "Status": "SAVED"
    }
) 

In [30]:
# Obtener los productos guardados
dt.query("12856333", "12856333#SAVED*", consumed_capacity=True)

Consumed Capacity: 0.5


[{'CreateTimestamp': '2023-06-14T23:38:57Z',
  'GSI1PK': '12856333#SAVED',
  'AccountId': '12856333',
  'ItemSKU': {'ProductId': 'P-8995', 'Category': 'Electronics'},
  'SK': '12856333#SAVED#P-8995',
  'Status': 'SAVED',
  'GSI1SK': '2023-06-14T23:38:57Z',
  'PK': '12856333'},
 {'CreateTimestamp': '2023-06-14T23:37:45Z',
  'GSI1PK': '12856333#SAVED',
  'AccountId': '12856333',
  'ItemSKU': {'ProductId': 'P-8997', 'Category': 'TV'},
  'SK': '12856333#SAVED#P-8997',
  'Status': 'SAVED',
  'GSI1SK': '2023-06-14T23:37:45Z',
  'PK': '12856333'}]

### Obtener todos los elementos comprados por un usuario

In [22]:
response = dt.query("12856333", "12856333#PURCHASED*")

In [27]:
for item in response:
    print(item['ItemSKU']['ProductId'], item['ItemSKU']['Quantity'], item['ItemSKU']['Price'], item['CreateTimestamp'])

P-10599 1 699.99 2023-06-15T15:19:38Z
P-8995 2 199.99 2023-06-15T15:19:38Z
P-8998 1 499.99 2023-06-15T15:19:38Z
P-10599 1 699.99 2023-06-14T23:29:19Z
P-8995 2 199.99 2023-06-14T23:29:19Z
P-8998 1 499.99 2023-06-14T23:29:19Z


## Cómo realizar análisis avanzados y crear visualizaciones de sus datos de Amazon DynamoDB mediante Amazon Athena

Puede obtener un enorme valor analítico de miles de millones de elementos y millones de solicitudes por segundo en su servicio de Amazon DynamoDB. Sin embargo, necesita exportar sus datos para obtener ese valor analítico. Copiar los datos de una tabla de DynamoDB a una plataforma de análisis le permite extraer información detallada. Para lograrlo, creemos que una canalización de big data bien diseñada ayuda a separar el procesamiento transaccional de los análisis. Esta publicación del blog muestra cómo crear una canalización de big data que transfiere los datos de la tabla de DynamoDB a Amazon S3. Esto le ayuda a realizar análisis avanzados mediante Amazon Athena, un servicio de consulta Presto totalmente administrado, y también le ayuda a crear visualizaciones y análisis ad hoc mediante Amazon QuickSight.

La mayoría de las aplicaciones de big data desacopladas tienen una canalización común que separa el almacenamiento de la informática, lo que le permite aprovechar las nuevas tecnologías de procesamiento a medida que llegan. El desacoplamiento permite el aprovisionamiento elástico de recursos informáticos para varios motores de análisis sin afectar a la durabilidad de los datos. También es posible que desee diseñar su canalización de modo que las etapas de almacenamiento y procesamiento se repitan para dar forma a los datos en un formato que las aplicaciones posteriores puedan consumir rápidamente.

Tres características principales influyen en el diseño de una canalización de big data:

* Latencia de la canalización general: ¿cuánto tiempo se necesita para pasar de los datos a la información? ¿Milisegundos, minutos o días?
* Rendimiento de los datos: ¿cuántos datos hay que ingerir y procesar? ¿Se trata de gigabytes, terabytes o petabytes?
* Coste: ¿cuál es el presupuesto previsto para su aplicación? La opción más rentable de AWS suele ser la correcta.

Otras consideraciones clave a la hora de diseñar su canalización de big data son la estructura de los datos, los patrones de acceso, la temperatura de los datos, la disponibilidad y durabilidad, y si el servicio está totalmente administrado. Utilizar las herramientas adecuadas para el trabajo en función de estas características es clave para una canalización de big data bien diseñada.

In [14]:
# Create Kinesis Data Stream
try:
    response = kinesis_client.create_stream(
        StreamName='CustomerPurchasesStream',
        ShardCount=1
    )
    print("Kinesis Data Stream created successfully.")
except ClientError as e:
    print(f"Error: {e.response['Error']['Message']}")  

In [20]:
# Get Kinesis Data Stream arn
response = kinesis_client.describe_stream(
    StreamName='CustomerPurchasesStream'
)
kinesis_arn = response['StreamDescription']['StreamARN']

In [24]:
# Asociate dynamodb table with kinesis data stream
try:
    response = dynamo_client.enable_kinesis_streaming_destination(
        TableName=dt.table_name,
        StreamArn=kinesis_arn
    )
    print("DynamoDB table associated with Kinesis Data Stream successfully.")
except ClientError as e:
    print(f"Error: {e.response['Error']['Message']}")

DynamoDB table associated with Kinesis Data Stream successfully.


In [32]:
df = pd.DataFrame()

In [69]:
def check_insert_data(records):
    all_items = []
    for elem in records:
            raw_kinesis_records = elem['kinesis']['data']
            payload=str(base64.b64decode(raw_kinesis_records).decode("UTF-8"))
            formattedPayload = json.loads(payload)
            ddb_data = json.loads(payload, parse_float=Decimal)
            # ------------------------------------------------------
            # PURCHASE EVENT
            if ddb_data['eventName'] == 'INSERT' and ddb_data['dynamodb']['NewImage']['Status']['S'] == 'PURCHASED':
                    item = dict()
                    item['eventId'] = ddb_data['eventID']
                    item['createTimestamp'] = ddb_data['dynamodb']['NewImage']['CreateTimestamp']['S']
                    item['productId'] = ddb_data['dynamodb']['NewImage']['ItemSKU']['M']['ProductId']['S']
                    all_items.append(item)
    return all_items

In [71]:
dm = check_insert_data(event['Records'])
df = pd.DataFrame(dm)
df

KeyError: 'M'